# La Familia Apply



Profesor: Mikel N. Legasa (legasam@unican.es)

### Introducción

`apply()` y sus variantes (`lapply()`, `sapply()`, `mapply()`...) son funciones para manipular matrices, arrays, listas y dataframes de manera repetitiva de distintas maneras, y permiten evitar el uso de ciclos `for` y `while`. Es decir, cuando se trata de aplicar de manera iterativa una función a una estructura de datos, en `R` esta serie de funciones permite hacerlo de manera concisa y rápida.

Comenzamos con un ejemplo sencillo, supongamos que queremos poner en minúsculas las letras de todos los nombres de un vector. La función de `R` que lo hace es `tolower()`. La forma habitual y primera aproximación para hacerlo es utilizar un ciclo for, como sigue:

In [1]:
nombres <- c("Paco","Álvaro","María", "Alex")
for (i in 1:length(nombres)){
    nombres[i] <- tolower(nombres[i])
}
nombres

[1] "paco"   "álvaro" "maría"  "alex"

Pero podemos escribirlo de manera más concisa usando la función `lapply()`:

In [2]:
nombres <- c("Paco","Álvaro", "María", "Alex")
nombres <- lapply(nombres, tolower)
nombres

[[1]]
[1] "paco"

[[2]]
[1] "álvaro"

[[3]]
[1] "maría"

[[4]]
[1] "alex"

Nota: Hay que tener en cuenta que muchas funciones de `R` están *vectorizadas*, lo que significa que pueden aplicarse directamente sobre vectores:

In [3]:
tolower(nombres)

[1] "paco"   "álvaro" "maría"  "alex"

Por supuesto, para cualquier función no vectorizada, o para navegar sobre tipos de datos más complejos, la familia Apply será muy útil.

En la siguiente sección veremos en detalle las diferencias entre las diferentes funciones de la familia, esencialmente cada una de ellas está indicada para operar sobre un tipo de estructura distinta. Nos centraremos en `apply()`, pensada para recorrer matrices sobre sus dimensiones; `lapply()`, pensada para recorrer listas y vectores; `sapply()`, que equivale a `lapply()` pero trata de simplificar en un array el resultado, y `mapply()` permite iterar sobre varias listas o vectores a la vez.

## `Apply()`

Esta primera función opera sobre arrays multidimensionales, en particular matrices. Sus tres principales argumentos son 

- `X`: Array sobre el que se desea operar.
- `MARGIN`: Dimensión sobre la que se desea aplicar la función indicada en `FUN`. Escribiendo `MARGIN = 1` aplicaremos la función a las filas de la matriz y escribiendo `MARGIN = 2` sobre las columnas. Notar que si `X` tiene más de dos dimensiones `MARGIN = 3` opera sobre la tercera dimensión, etc...
- `FUN`: Función a aplicar.


Vamos a trabajar con la matriz `A`:

In [3]:
?sample

In [4]:
set.seed(1) # resultados reproducibles
A <- replicate(expr = sample(1:15, size = 6),n = 6) # replicate() evalua la expresión expr n veces
A

4,15,11,6,5,8
6,10,6,11,6,9
8,9,15,13,1,7
11,1,14,3,15,3
3,3,8,8,10,10
9,2,10,2,4,13


**Ejemplo:** calcular el máximo de cada fila de la matriz:

In [5]:
apply(X = A, MARGIN = 1, FUN = max)

[1] 15 11 15 15 10 13

**Ejemplo:** calcular el mínimo de cada columna de la matriz:

In [6]:
apply(X = A, MARGIN = 2, FUN = min)

[1] 3 1 6 2 1 3

`apply()` también permite pasar argumentos adicionales a la función especificada en `FUN`. Es decir, en nuestro último ejemplo `apply()` aplica a las columnas de la matriz `A` la función `min()`, para la cual podemos querer especificar algún argumento adicional (las columnas de `A` entran como primer argumento). En efecto, `min()` tiene el argumento `na.rm`, que permite ignorar los `NA` en el cálculo del mínimo, si los hubiera. Veamos un ejemplo:

**Ejemplo:** Calcular el mímimo de cada columna de la matriz `A`, a la que hemos introducido dos `NA`:

In [7]:
# Introducimos unos NA
A[1,2] <- NA
A[3,2] <- NA
A[4,6] <- NA

apply(X = A, MARGIN = 2, FUN = min)

[1]  3 NA  6  2  1 NA

Como vemos, al haber `NA`s en la matriz, el mínimo de alguna columna es `NA` también. Si queremos evitarlo, debemos pasarle el argumento `na.rm = TRUE` a la función `min()`. Esto se hace pasándoselo directamente a `apply()` (Si consultas la ayuda (`?apply`), puedes ver el argumento `...`, argumentos opcionales para `FUN`):

In [8]:
apply(X = A, MARGIN = 2, FUN = min, na.rm = TRUE)

[1] 3 1 6 2 1 7

También podemos usar `apply()` sobre matrices de más dimensiones, por ejemplo:

**Ejemplo:** Calcular la media para de las submatrices sobre la dimensión $3$:

In [9]:
A3 <- replicate(replicate(expr = sample(1:15, size = 5),n = 5), n= 10) # A3 es de dimensiones 5x5x10
print(A3[,,10]) # ejemplo
apply(A3, MARGIN = 3, FUN = mean)

     [,1] [,2] [,3] [,4] [,5]
[1,]    5    1    2    4    8
[2,]    4   11    6    3   13
[3,]   11    9    3    7    5
[4,]    9    3    4   10    1
[5,]    2   15   13    1    2


[1] 7.80 8.28 7.96 7.12 7.56 9.16 7.68 7.40 7.24 6.08

Hay que tener en cuenta que podemos escribir nuestra propia función dentro de la función `apply()`, lo que será muy útil para funciones que requieran algo más de complejidad. Es decir, el ejemplo anterior equivale a 

In [10]:
apply(A3, MARGIN = 3, FUN = function(x) {return(mean(x))})

[1] 7.80 8.28 7.96 7.12 7.56 9.16 7.68 7.40 7.24 6.08

**Ejercicio:** Obtén la media por filas de la matriz `A`:

In [13]:
apply(A, MARGIN = 1, mean, na.rm = TRUE)

[1] 6.800000 8.000000 8.800000 8.800000 7.000000 6.666667

**Ejercicio:** En la matriz `A`, queremos saber cuantos `NA` hay en cada columna (recordar funciones `sum()` e `is.na()`):

In [15]:
apply(A, MARGIN = 2, FUN = function(x) {sum(is.na(x))})

[1] 0 2 0 0 0 1

**Ejercicio:** Obtén la matriz resultado de sumar a cada columna de la matriz `A` el vector `y`:

In [16]:
y <- seq(1,6)
apply(A, MARGIN = 2, FUN = function(x) {return(x + y)})

5,NA,12,7,6,9
8,12,8,13,8,11
11,NA,18,16,4,10
15,5,18,7,19,NA
8,8,13,13,15,15
15,8,16,8,10,19


`apply()` puede utilizarse también sobre `data.frame`s, tratándolos como si fuera una matriz.

**Ejercicio**: Del `data.frame` `alumnos`, verifica si hay alguien que se llame "jose" e identifica la clase en la que se encuentra, utilizando la función `is.element()` y definiendo una función dentro de `apply()`, después haz lo mismo sin definir una función:

In [17]:
alumnos <-
data.frame(fisica = c("juan", "rodrigo", "maria", "jose"),
           matematicas = c("joaquin", "maialen", "jose", NA),
           biologia = c("ana", "daniel", "markel", "adriana"))

apply(alumnos, MARGIN = 2, is.element, el='jose')
apply(alumnos, MARGIN = 2, FUN = function(x) {return(is.element('jose', x))})

fisica matematicas    biologia 
       TRUE        TRUE       FALSE

fisica matematicas    biologia 
       TRUE        TRUE       FALSE

Por último, nótese que `apply()` puede producir resultados inesperados. La ayuda de `apply()` especifica "Si la llamada a la función devuelve un vector de longitud $n$, entonces `apply()` devuelve un array de dimensión `c(n, dim(X)[MARGIN])`. Compruébalo:

**Ejercicio:** Obtén la **matriz resultado** de sumar a cada **fila** de la matriz `A3` el vector `y`:

In [18]:
y <- seq(1,5)
dim(apply(A3, MARGIN = 1, function(x) {return(x+y)}))

[1] 50  5

Nota: En `R` ya existen las funciones `rowMeans()`, `rowSums()`, `colMeans()`, `colSums()`. Éstas están ya precompiladas en `C` y son mucho más rápidas que sus equivalentes utilizando bien `apply()` o ciclos for.

## `lapply()` y `sapply()`

`lapply()` es la función indicada para trabajar con listas. También permite iterar sobre vectores, como vimos en la introducción, y sobre `data.frame`s. Sus dos argumentos principales son, análogamente a `apply()`,

- `X`: Lista sobre la que se desea iterar.
- `FUN`: Función a aplicar.

E idénticamente a ésta, también permite especificar argumentos adicionales (`...`).

**Ejercicio:** De la lista siguiente obtén la media de cada vector

In [19]:
lista.vec <- list(c(9,10,5,6), 
                    c(9,9,8,4),
                    c(4,3,6,6))
lapply(lista.vec, mean)

[[1]]
[1] 7.5

[[2]]
[1] 7.5

[[3]]
[1] 4.75

**Ejercicio:** De la lista de matrices siguientes, obtén las medias por columnas de cada matriz.

In [20]:
lista.mat <- replicate(replicate(expr = sample(1:15, size = 6),n = 6,), n = 4 , simplify = F) 
# simplify = FALSE no simplifica en un array

lapply(lista.mat, colMeans)

[[1]]
[1] 8.166667 8.166667 6.166667 6.500000 9.500000 7.333333

[[2]]
[1] 10.500000  9.500000  9.000000  9.666667  5.333333  7.666667

[[3]]
[1]  6.666667  8.333333  8.000000  9.000000  8.500000 10.166667

[[4]]
[1]  5.833333  9.500000  8.000000 10.166667  7.666667  9.000000

Debe tenerse en cuenta que: 

1. El objeto devuelto siempre es una lista, incluso aunque el objeto sobre el que iteramos no lo sea. Lo podemos ver con nuestro ejemplo inicial:

In [21]:
is.list(lapply(nombres, tolower))

[1] TRUE

2. Si se aplica `lapply()` a una matriz esta tratará como un vector, i.e. elemento a elemento.

**Ejemplo:**

In [22]:
lapply(A, mean)

[[1]]
[1] 4

[[2]]
[1] 6

[[3]]
[1] 8

[[4]]
[1] 11

[[5]]
[1] 3

[[6]]
[1] 9

[[7]]
[1] NA

[[8]]
[1] 10

[[9]]
[1] NA

[[10]]
[1] 1

[[11]]
[1] 3

[[12]]
[1] 2

[[13]]
[1] 11

[[14]]
[1] 6

[[15]]
[1] 15

[[16]]
[1] 14

[[17]]
[1] 8

[[18]]
[1] 10

[[19]]
[1] 6

[[20]]
[1] 11

[[21]]
[1] 13

[[22]]
[1] 3

[[23]]
[1] 8

[[24]]
[1] 2

[[25]]
[1] 5

[[26]]
[1] 6

[[27]]
[1] 1

[[28]]
[1] 15

[[29]]
[1] 10

[[30]]
[1] 4

[[31]]
[1] 8

[[32]]
[1] 9

[[33]]
[1] 7

[[34]]
[1] NA

[[35]]
[1] 10

[[36]]
[1] 13

3. Si se aplica sobre un `data.frame` éste se iterará por columnas.

**Ejercicio:** Del `data.frame` `alumnos`, verifica si hay alguien que se llame "jose" e identifica la clase en la que se encuentra, esta vez utilizando `lapply()`:

In [23]:
lapply(alumnos, function(x) {return(is.element('jose', x))})

$fisica
[1] TRUE

$matematicas
[1] TRUE

$biologia
[1] FALSE

**Ejercicio:** La lista siguiente no es homogénea. Verifica elemento a elemento si el segundo elemento de la sublista es un `data.frame`. Si no lo es, conviértelo en uno y marca la primera variable de cada lista como `"ndf"`. Si lo es, márcala como `"df"`.

In [25]:
lista.vars <-list(
             list("df", as.data.frame(replicate(sample(1:15, 6),n = 6))),
             list("var3", replicate(sample(1:15, 6),n = 6)),
             list("t", as.data.frame(replicate(sample(1:15, 6),n = 6))),
             list("loc", replicate(sample(1:15, 6),n = 6))
             )

lapply(lista.vars, function(x) {
    if (is.data.frame(x[[2]]))
        return(list('df', x[[2]]))
    else
        return(list('ndf', as.data.frame(x[[2]])))
    return()
})

V1,V2,V3,V4,V5,V6
13,6,2,12,6,15
8,14,8,10,7,14
9,9,13,11,11,5
5,1,9,8,12,13
10,7,7,6,2,11
14,5,3,5,8,2
V1,V2,V3,V4,V5,V6
15,3,9,8,7,14
5,5,2,14,5,5
7,9,5,12,2,13


---
#### `sapply()` 

La función `sapply()` es idéntica a `lapply()`, pero trata de simplificar el resultado a un vector, matriz o array. Si la salida es de una dimensión, lo convertirá a un vector, si es una matriz a una matriz (utiliza internamente la función `simplify2array()`). Como **ejemplo:**

In [39]:
is.vector(sapply(nombres, tolower))

[1] TRUE

In [40]:
is.list(sapply(nombres, tolower))

[1] FALSE

`sapply()` tiene dos argumentos adicionales respecto a `lapply()`:
- `simplify` Si se especifica `simplify = FALSE`, entonces es idéntico a `lapply()`.
- `USE.NAMES` Si se especifica `USE.NAMES = TRUE` (por defecto), y `X` es de tipo `character`, se usan estos como nombres para el output.

**Ejercicio:** Verifica el output de los dos ejercicios anteriores cuando en vez de `lapply()` se utiliza `sapply()`

In [171]:
aux <- sapply(alumnos, function(x) {return(is.element('jose', x))})
aux
is.vector(aux)

fisica matematicas    biologia 
       TRUE        TRUE       FALSE

[1] TRUE

In [52]:
sapply(lista.vars, function(x) {
    if (is.data.frame(x[[2]]))
        return(list('df', x[[2]]))
    else
        return(list('ndf', as.data.frame(x[[2]])))
    return()
})

df,ndf,df,ndf
"13, 8, 9, 5, 10, 14, 6, 14, 9, 1, 7, 5, 2, 8, 13, 9, 7, 3, 12, 10, 11, 8, 6, 5, 6, 7, 11, 12, 2, 8, 15, 14, 5, 13, 11, 2","15, 5, 7, 13, 8, 14, 3, 5, 9, 1, 10, 15, 9, 2, 5, 8, 3, 7, 8, 14, 12, 5, 4, 9, 7, 5, 2, 1, 8, 15, 14, 5, 13, 3, 9, 12","1, 13, 9, 12, 8, 14, 6, 15, 8, 2, 3, 14, 12, 13, 5, 10, 1, 7, 6, 9, 12, 3, 5, 7, 12, 8, 11, 7, 6, 5, 6, 2, 4, 13, 9, 8","3, 8, 14, 7, 5, 11, 1, 13, 2, 7, 10, 6, 7, 8, 11, 3, 13, 5, 14, 13, 10, 9, 8, 1, 7, 8, 13, 9, 14, 5, 2, 11, 6, 13, 1, 15"


## `mapply()`

Hasta ahora hemos visto cómo vectorizar sobre una lista, matriz, vector. `mapply()`  (*multivariate apply*) permite vectorizar más de un argumento, es decir, aplicar una función sobre múltiples argumentos de manera vectorizada. 

Sus argumentos son:
- `FUN`,	Función a aplicar
- `...` , Argumentos sobre los que vectorizar.
- `MoreArgs`, Lista de argumentos adicionales (no vectorizados).
- `SIMPLIFY`, Si se marca como `TRUE`, se intentará *simplificar* el resultado a un vector, matriz o array multidimensional. Funciona análogamente al simplificado de `sapply()`

Veamos como funciona con un **ejemplo**:

En este caso tenemos dos listas de vectores, y queremos sumar vector a vector. Podemos hacerlo con `mapply()`:

In [53]:
lista.vec1 <- list(c(1,1,10,20), c(-1,12,122,44), c(34,12,65,23), c(1,1,2,4))
lista.vec2 <- list(c(100,200,100,210), c(400,120,1220,104), c(-530,-320,630,-650), c(-1,-1,2,-4))
mapply(FUN = function(x,y) x+y, lista.vec1, lista.vec2)

101,399,-496,0
201,132,-308,0
110,1342,695,4
230,148,-627,0


Por defecto `mapply()` simplifica el resultado en una matriz. Podemos evitarlo si especificamos `SIMPLIFY = FALSE`:

In [54]:
mapply(FUN = function(x,y) x+y, lista.vec1, lista.vec2, SIMPLIFY = FALSE)

[[1]]
[1] 101 201 110 230

[[2]]
[1]  399  132 1342  148

[[3]]
[1] -496 -308  695 -627

[[4]]
[1] 0 0 4 0

La manera de asignar argumentos adicionales es mediante el argumento `MoreArgs` (Cuidado, debe ser una lista). 

**Ejemplo:** Supongamos que tenemos una función que suma y admite un tercer argumento `absoluto`, que hace que sume y tome el valor absoluto si éste es igual a `TRUE`. Queremos especificarlo como `TRUE`, para todas las sumas que haga la función:

In [55]:
mapply(FUN = function(x,y, absoluto = FALSE) {
                if (absoluto) return(abs(x+y))
                else return(x+y)
             },
       lista.vec1, lista.vec2, SIMPLIFY = FALSE, MoreArgs = list(absoluto = TRUE)
      )

[[1]]
[1] 101 201 110 230

[[2]]
[1]  399  132 1342  148

[[3]]
[1] 496 308 695 627

[[4]]
[1] 0 0 4 0

**Ejercicio:** De la lista `mi.lista.1`, obtén del vector $1$ el elemento $5$, del $2$ el elemento $4$, del $3$ el elemento $3$, del $4$ el $2$ y del $5$ el $1$:

In [56]:
mi.lista.1 <- replicate(sample(1:10, size = 5, replace = T),n = 5, simplify = F)

mapply(FUN = function(x, pos) x[pos],
       mi.lista.1, c(5, 4, 3, 2, 1))

[1] 10  4  8  8  1

**Ejercicio:** Las listas `mi.lista.1` y `mi.lista.2` contienen el mismo número de vectores. Obtén el máximo elemento de la combinación de los pares de matrices, i.e. si `mi.lista.1 = {v11, v12, ...}`, `mi.lista.2 = {v21, v22, ...}`, obtén `{max(max(v11,v21)), max(max(v12,v22)), ...}`

In [58]:
mi.lista.2 <- replicate(sample(1:100, size = 4, replace = T),n = 5, simplify = F)

mapply(FUN = function(x, y) max(c(x, y)),
       mi.lista.1, mi.lista.2)

[1] 48 84 91 97 87

**Ejercicio:** Repite el ejercio anterior. Esta vez los vectores pueden tener `NA`s y queremos ignorarlos:

In [61]:
mi.lista.1[[1]][3] <- NA
mi.lista.2[[5]][4] <- NA

mapply(FUN = function(x, y) max(c(x, y), na.rm = T),
       mi.lista.1, mi.lista.2)

[1] 48 84 91 97 78

**Ejercicio:** Repite el ejercicio anterior sin utilizar ciclos ni `mapply()` (Pista: por ejemplo con dos `lapply()` un `cbind()` y un `apply()`

In [66]:
apply(X = matrix(unlist(cbind(lapply(mi.lista.1, max, na.rm = T),
            lapply(mi.lista.2, max, na.rm = T))), nrow=5, ncol=2),
    MARGIN = 1, FUN = function(x) max(x))

[1] 48 84 91 97 78

**Ejercicio:** Verifica usando `mapply()`, del data.frame alumnos de más atrás, si en la primera clase hay alguien que se llame "rodrigo", en la segunda alguien que se llame "maialen" y en la tercera alguien que se llame "mikel":

In [68]:
mapply(FUN = function(x, val) return(is.element(val, x)), 
       alumnos, c('rodrigo', 'maialen', 'mikel'))

fisica matematicas    biologia 
       TRUE        TRUE       FALSE

**Ejercicio:** Se ha intentado aplicar la función `mi.suma` a las dos listas para sumar en valor absoluto los vectores. ¿Por qué falla la siguiente instrucción? Arregla la instrucción `mapply()` (sin tocar `mi.suma()`):

In [69]:
mi.suma <- 
    function(x, y, z = "Sumando...") {
               print(z)
               return(abs(x + y))
             }

mapply(mi.suma,
       lista.vec1, lista.vec2, MoreArgs = list(z = 'Add...'), SIMPLIFY = F
      )

[1] "Add..."
[1] "Add..."
[1] "Add..."
[1] "Add..."


[[1]]
[1] 101 201 110 230

[[2]]
[1]  399  132 1342  148

[[3]]
[1] 496 308 695 627

[[4]]
[1] 0 0 4 0

## Otras funciones: `rapply()`, `tapply()` y `vapply()`

Existen más funciones en la familia Apply diseñadas para tareas más específicas. En esta sección cubriremos algunas de ellas someramente con un par de ejemplos. La primera, `rapply()`, está pensada para operar **recursivamente** sobre listas. Es especialmente útil cuando nos encontramos con listas mal formateadas que contienen sublistas mezcladas con, por ejemplo, números:

**Ejemplo:** La siguiente lista contiene números y listas de números, en lugar de ser una lista de números. Supongamos que queremos asignar un `NA` a todos los elementos de la lista que sean negativos, observa lo que ocurre si se utiliza `lapply()`:

In [70]:
lista.e <- list(1,2,-1, list(10,12,-1), 4,-4, list(1,1), list(-1,-1,-2,5),3,6,1 ,-1)
lapply(lista.e, function(x){
                    if(x<0) return(NA)
                    else return(x)
    } )

Warning message in if (x < 0) return(NA) else return(x):
“the condition has length > 1 and only the first element will be used”Warning message in if (x < 0) return(NA) else return(x):
“the condition has length > 1 and only the first element will be used”Warning message in if (x < 0) return(NA) else return(x):
“the condition has length > 1 and only the first element will be used”

[[1]]
[1] 1

[[2]]
[1] 2

[[3]]
[1] NA

[[4]]
[[4]][[1]]
[1] 10

[[4]][[2]]
[1] 12

[[4]][[3]]
[1] -1


[[5]]
[1] 4

[[6]]
[1] NA

[[7]]
[[7]][[1]]
[1] 1

[[7]][[2]]
[1] 1


[[8]]
[1] NA

[[9]]
[1] 3

[[10]]
[1] 6

[[11]]
[1] 1

[[12]]
[1] NA

`rapply()` permite precisamente operar de manera recursiva sobre la lista sin preocuparnos del mal formato:

In [71]:
rapply(lista.e, function(x){
                    if(x<0) return(NA)
                    else return(x)
                })

[1]  1  2 NA 10 12 NA  4 NA  1  1 NA NA NA  5  3  6  1 NA

No sólo eso, también permite especificar el tipo de argumento sobre el que se quiere operar utilizando el comando `classes`. Por ejemplo, podemos encontrarnos con un vector que contenga números con algún caracter, etc... Y querer solamente aplicar una función a los números:

In [73]:
lista.e2 <- list(2, 1, 2, NA, 12, "err", list("err", 1, 3), 5, NA, list(), NULL, 2)
rapply(lista.e2, f = function(x) x^2, classes = "numeric")

[1]   4   1   4 144   1   9  25   4

**Ejercicio:** Eleva al cuadrado los números de la lista `lista.e2` y devuelve la misma estructura que arriba, sin usar `rapply()`. Usa `lapply()` o `sapply()`:

In [78]:
sapply(lista.e2, FUN = function(x) {
    if (is.numeric(x)) return(x^2)
    if (is.list(x)) return(sapply(x, FUN = function(y) {
                            if (is.numeric(y)) return(y^2)}))
    if(!is.numeric(x)) return(NA)
    })

[[1]]
[1] 4

[[2]]
[1] 1

[[3]]
[1] 4

[[4]]
[1] NA

[[5]]
[1] 144

[[6]]
[1] NA

[[7]]
[[7]][[1]]
NULL

[[7]][[2]]
[1] 1

[[7]][[3]]
[1] 9


[[8]]
[1] 25

[[9]]
[1] NA

[[10]]
list()

[[11]]
[1] NA

[[12]]
[1] 4

**Ejercicio:** Vuelve a hacerlo, esta vez sin usar `rapply()`, `lapply()` ni `sapply()`, i.e. usa un ciclo:

In [102]:
result <- c()
for (i in 1:length(lista.e2)) {
    val <- lista.e2[[i]]
    if (is.numeric(val))
        result <- c(result, val^2)
    else if (is.list(val) && length(val) >= 1) {
        for (j in 1:length(val)) {
            val_in <- lista.e2[[i]][[j]]
            if (is.numeric(val_in))
                result <- c(result, val_in^2)
        }
    }
}
result

[1]   4   1   4 144   1   9  25   4

---

Otra función útil es `tapply()`, indicada cuando hay que aplicar una función a una variable, dividiendo ésta en grupos según dicta otra variable. La variable sobre la que iterar es, como viene siendo habitual, `X`, y el argumento por el que queremos agrupar es `INDEX`.

Veámoslo con un **ejemplo** del dataset `iris`. El objetivo es obtener la media de la variable `iris$Sepal.Length` pero agrupando por las diferentes especies (`iris$Species`), i.e. obtener la media de la variable $Sepal.Length$ para cada especie:

In [108]:
media.Sepal.Length.por.Species <-
    tapply(X = iris$Sepal.Length, INDEX = iris$Species, FUN = mean) # ignora el error, es un error de jupyter, no de R. El resultado es correcto
t(media.Sepal.Length.por.Species)

setosa,versicolor,virginica
5.006,5.936,6.588


---

La última función que veremos es `vapply()`. Esta función es similar a `sapply()`, con la diferencia de que es posible controlar la clase del output, razón por la cual esta función es considerada más robusta que `sapply()`, ya que evita resultados inesperados a la hora de trabajar con estructuras complejas. No sólo eso, también puede ser ligeramente más eficiente, puesto que no tiene que decidir la estructura de salida sinó que viene dada. Veámoslo con un par de **ejemplos:**

Queremos, de `lista1`, devolver `TRUE` si la sublista contiene alguna `e`, y `FALSE` si no. Programamos nuestra función `busca.e`:

In [114]:
1 in c(1, 3)

ERROR: Error in parse(text = x, srcfile = src): <text>:1:3: unexpected 'in'
1: 1 in
      ^


In [121]:
lista1 <- list( c("a","e","i","o","u"), c("m","n","l","r"), c("f","e","r","m","e"))
busca.e <- function(x) x=="e"

Buscamos un vector de tres elementos que contenga `TRUE` si la lista contenía alguna `"e"` y `FALSE` si no.

In [122]:
sapply(lista1, busca.e )

[[1]]
[1] FALSE  TRUE FALSE FALSE FALSE

[[2]]
[1] FALSE FALSE FALSE FALSE

[[3]]
[1] FALSE  TRUE FALSE FALSE  TRUE

Sin embargo, como `busca.e` no la hemos hecho del todo bien, obtenemos una lista que no es lo que esperamos. Es más robusto controlar la estructura especificándole que nos devuelva específicamente un objeto de tipo `logical` de longitud 1, lo que puede hacerse con `vapply()`:

In [123]:
busca.e <- function(x) return(is.element("e", x))
vapply(lista1, busca.e, logical(1))

[1]  TRUE FALSE  TRUE

In [124]:
busca.e <- function(x) if (is.element("e", x)) return('ok')
vapply(lista1, busca.e, logical(1))

ERROR: Error in vapply(lista1, busca.e, logical(1)): values must be type 'logical',
 but FUN(X[[1]]) result is type 'character'


`vapply()` comprueba el output y nos avisa de que estamos intentando devolver resultados de longitud $5$ en lugar de resultados de longitud $1$ (*`values must be length 1`*). Podemos verlo con otro ejemplo. Queremos multiplicar a los elementos de `mi.lista3` por $100$. 

Esperamos que la entrada sea una lista de números, pero alguien ha programado mal la función de la que vienen los datos y no lo es, ya que contiene también vectores:

In [125]:
mi.lista3 <- list(1,3.14,1.51,2.111,c(-1,-404,-1),14)
vapply(mi.lista3, FUN = function(x) x*100, FUN.VALUE = 1)

ERROR: Error in vapply(mi.lista3, FUN = function(x) x * 100, FUN.VALUE = 1): values must be length 1,
 but FUN(X[[5]]) result is length 3


`vapply()` nos avisa de nuevo de que hay un error en los datos. Obsérvese como si lo aplicamos a la siguiente lista, que si tiene el formato esperado, el resultado es correcto:

In [126]:
mi.lista4 <- list(1,3.14,1.51,2.111,1,404,1,14)
vapply(mi.lista4, FUN = function(x) x*100, FUN.VALUE = 1)

[1]   100.0   314.0   151.0   211.1   100.0 40400.0   100.0  1400.0

### Nota Final

Utilizar las funciones de la familia Apply permite escribir código de `R` de manera concisa y eficiente. Se debe tener en cuenta que utilizarlas no es *per se* más rápido que utilizar un ciclo `for`. En ese sentido, `apply()` y sus variantes simplemente encapsulan el ciclo. (Existen casos en los que sí son más eficientes, por ejemplo, cuando hay que entrelazar varios cilos for). La razón por la que son habitualmente y en la práctica más rápidas que un ciclo `for` en `R` es porque éstas reservan espacio en memoria de antemano de manera implícita. Si se hace esto manualmente y se programa correctamente el ciclo `for`, puede comprobarse que los tiempos de ejecución son similares.

Nótese por último que si existe una función vectorizada para la operación que estuvieras intendando hacer, sea con `apply()` o un ciclo, deberías usar dicha función. Por ejemplo, calcular la media por columnas de una matriz puede hacerse con un ciclo y mediante el comando `apply(foo, 2, mean)`, pero es más eficiente si se utiliza la función de `colMeans(foo)`.

Por último, las funciones de la familia Apply abren la puerta a **paralelizar el proceso** de manera inmediata.



# Paralelizando con la librería *parallel*

Cualquier ciclo que programemos se ejecuta de manera secuencial, lo que implica que se está usando un solo procesador. Paralelizar implica subdividir la tarea en subtareas para que sean procesadas por los diferentes núcleos (o procesadores, si nuestra máquina tiene más de uno).

Las funciones y metodología que hemos visto hasta ahora permiten paralelizar procesos de manera muy sencilla. Precisamente porque se está repitiendo una tarea múltiples veces de manera independiente, todas ellas son de naturaleza paralelizable. Para hacerlo, basta tener en cuenta que, a excepción de `mapply()`, las funciones que hemos visto tienen su equivalente en la librería parallel: `parApply()` `parLapply()`, `parSapply()`.  

Funcionan exactamente igual, pero requieren como primer argumento un objeto de tipo *cluster* de `parallel`, que podemos iniciar con la función `makeCluster()`.

En Mac/Linux la sintaxis básica para iniciar este objeto es `makeCluster(numero_nodos, type="FORK")`, que inicia un *cluster* de tipo *FORK* con `numero_nodos` nodos (la cantidad de *workers* para el proceso a paralelizar).

Este tipo de *cluster* no está disponible en Windows, con lo que se debe usar un cluster de tipo *PSOCK* (Parallel Socket Cluster), con `type = PSOCK`. La diferencia principal reside en que con *FORK* todas las variables y funciones quedan automáticamente exportadas al *cluster*, con lo que no tenemos que preocuparnos de hacerlo manualmente.

Vamos a ilustrar esto con un **ejemplo**. Además de la librería `parallel`, también vamos a cargar la librería `microbenchmark`, para hacer pruebas de rendimiento:

In [127]:
library(parallel) 
library(microbenchmark)

ERROR: Error in library(microbenchmark): there is no package called ‘microbenchmark’


La función `detectCores()` permite ver la cantidad de hilos disponibles en el sistema:

In [128]:
detectCores()

[1] 20

Lo ideal es siempre dejar libre al menos uno para el sistema, de modo que la práctica más habitual es utilizar como número de nodos `detectCores()-1`, 

In [129]:
cl <- makeCluster(spec = detectCores()-1, type = "FORK") # iniciamos el cluster
cl

socket cluster with 19 nodes on host ‘localhost’

Vamos a coger uno de los ejemplos iniciales y utilizar la paralelización (Creamos una lista de matrices algo más grandes):

In [130]:
A.list <- replicate(replicate(expr = sample(1:50, size = 50, replace = T),n = 50), n = 100, simplify = F) # matriz 500x500

In [131]:
cmean <- parLapply(cl = cl, X = A.list,  mean) # media de cada matriz
# equivale a 
# cmean <- lapply(X = A.list,  mean) # media de cada matriz

Conviene tener en cuenta que no todo es ideal paralelizarlo. De hecho, la operación que acabamos de hacer no tiene sentido a nivel de rendimiento. Comprobemos el rendimiento de `parApply()` contra `apply()`:

In [31]:
mbp <- microbenchmark(parLapply(cl = cl, X = A.list,  mean), times = 10)
mb <- microbenchmark(lapply(X = A.list,  mean), times = 10)

In [32]:
mean(mbp$time)/10^9 # segundos

[1] 0.05719618

In [33]:
mean(mb$time)/10^9 # segundos

[1] 0.001293796

Siempre que acabemos de paralelizar hay que deterner el cluster que creamos, utilizando `stopCluster()`:

In [132]:
stopCluster(cl)

 Nota: La función `parMapply()` no existe, por lo que si se quiere paralelizar un `mapply()` se debe acudir a `mcmapply()`. También existe `mclapply()`, que funciona análogamente a `lapply()`. En éstas no hay que iniciar el *cluster* de antemano, pero no están disponibles en Windows.
 
 Usando `mclapply()` paralelizar es tan sencillo como:

In [133]:
mclapply(X = A.list, FUN = mean, mc.cores = 19)

[[1]]
[1] 25.0312

[[2]]
[1] 25.944

[[3]]
[1] 25.5224

[[4]]
[1] 25.622

[[5]]
[1] 25.3436

[[6]]
[1] 25.6416

[[7]]
[1] 25.0244

[[8]]
[1] 25.8908

[[9]]
[1] 25.152

[[10]]
[1] 25.1832

[[11]]
[1] 25.5844

[[12]]
[1] 25.1996

[[13]]
[1] 25.7324

[[14]]
[1] 25.9124

[[15]]
[1] 25.35

[[16]]
[1] 25.5828

[[17]]
[1] 25.9316

[[18]]
[1] 25.07

[[19]]
[1] 25.9044

[[20]]
[1] 25.7932

[[21]]
[1] 24.9112

[[22]]
[1] 25.7952

[[23]]
[1] 25.9232

[[24]]
[1] 25.4248

[[25]]
[1] 25.8092

[[26]]
[1] 25.5748

[[27]]
[1] 25.3124

[[28]]
[1] 25.4596

[[29]]
[1] 25.9028

[[30]]
[1] 25.0372

[[31]]
[1] 25.5088

[[32]]
[1] 25.1492

[[33]]
[1] 24.8796

[[34]]
[1] 25.5432

[[35]]
[1] 25.4932

[[36]]
[1] 25.568

[[37]]
[1] 25.0192

[[38]]
[1] 25.842

[[39]]
[1] 25.552

[[40]]
[1] 24.8688

[[41]]
[1] 25.124

[[42]]
[1] 25.8008

[[43]]
[1] 25.2272

[[44]]
[1] 25.1452

[[45]]
[1] 25.842

[[46]]
[1] 25.01

[[47]]
[1] 25.5808

[[48]]
[1] 25.5988

[[49]]
[1] 25.6576

[[50]]
[1] 25.4872

[[51]]
[1] 25.5928

[[52]]
[1] 25.3264

[[53]]
[1] 25.1844

[[54]]
[1] 25.8176

[[55]]
[1] 25.0792

[[56]]
[1] 25.564

[[57]]
[1] 25.5784

[[58]]
[1] 25.6572

[[59]]
[1] 25.4088

[[60]]
[1] 25.718

[[61]]
[1] 25.4664

[[62]]
[1] 25.4608

[[63]]
[1] 25.4524

[[64]]
[1] 25.4516

[[65]]
[1] 24.9268

[[66]]
[1] 25.504

[[67]]
[1] 24.992

[[68]]
[1] 25.5988

[[69]]
[1] 25.2804

[[70]]
[1] 25.7212

[[71]]
[1] 25.4316

[[72]]
[1] 25.964

[[73]]
[1] 25.7888

[[74]]
[1] 25.72

[[75]]
[1] 25.9016

[[76]]
[1] 26.0328

[[77]]
[1] 25.8464

[[78]]
[1] 25.4896

[[79]]
[1] 25.3048

[[80]]
[1] 25.8012

[[81]]
[1] 25.4636

[[82]]
[1] 25.2628

[[83]]
[1] 25.186

[[84]]
[1] 25.4148

[[85]]
[1] 25.7456

[[86]]
[1] 25.8836

[[87]]
[1] 26.2712

[[88]]
[1] 25.5764

[[89]]
[1] 25.7564

[[90]]
[1] 25.814

[[91]]
[1] 25.888

[[92]]
[1] 25.2436

[[93]]
[1] 25.5868

[[94]]
[1] 25.532

[[95]]
[1] 25.018

[[96]]
[1] 25.4924

[[97]]
[1] 25.4892

[[98]]
[1] 24.8556

[[99]]
[1] 25.6464

[[100]]
[1] 25.6508

Nótese que `mclapply()` abre y cierra el *cluster* automáticamente.

### ¿Cuándo paralelizar?

Aunque en teoría cada procesador debería reducir linealmente el tiempo de cómputo, en la práctica hay trabajo adicional que reduce la eficiencia. Tanto el código como los datos necesitan ser transferidos a cada núcleo y se deben crear los subprocesos, lo que suma tiempo. Es por ello que vemos que paralelizar en el ejemplo de arriba no merece la pena, ya que si el tiempo de computación es muy corto para cada subtarea, el coste de preparar los recursos para la paralelización es más grande que el ahorro conseguido. 

A continuación vamos a ver un ejemplo de un caso en el que la paralelización disminuye sustancialmente el tiempo de computación. Tenemos una lista de 19 vectores de tamaño 1000 con números obtenidos de una distribución uniforme, `unif`, al que queremos aplicar la función `mi.funcion()`, que definimos a continuación:

In [36]:
unif <- replicate(runif(n = 1000), n = 19, simplify = F)
mi.funcion <- function(x) {
    sort(x)
    d.tipica <- sd(x)
    media <- mean(x)
    
    error.rel.media <- 1
    error.rel.d.tipica <- 1
    while(error.rel.media > 0.01 || error.rel.d.tipica > 0.001){
        muestra <- sample(x, size = 10)
        error.rel.media <- abs(mean(muestra) - media)/media
        error.rel.d.tipica <- abs(sd(muestra) - d.tipica)/d.tipica
    }
    return(muestra)
}  

La función `mi.funcion()` extrae muestras de $10$ elementos de cada vector hasta que obtiene una con la media y desviación típica razonablemente cerca.

**Ejercicio:** Utiliza la función `microbenchmark()` para medir el tiempo de ejecución de aplicar la función `mi.funcion` a cada elemento de la lista `unif`, primero sin paralelizar y después paralelizando.